## Imports

In [1]:
using Random
using StableRNGs
using StatsBase
using Test

## Abstract Types

In [2]:
# top level
abstract type AbstractEntity end

# entity that cannot be composed into subentities
abstract type AbstractAtom <: AbstractEntity end

abstract type AbstractDescription end

# datatype of the "DNA" or information for encoding an entity
abstract type AbstractGenotype <: AbstractDescription end

# datatype of the "expression" of the genotype, used for evaluation
abstract type AbstractPhenotype <: AbstractDescription end

# atomic entity comprising a genotype and a phenotype
abstract type AbstractOrganism{G <: AbstractGenotype, P <: AbstractPhenotype} <: AbstractAtom end

# a set of organisms (which itself constitutes an entity)
abstract type AbstractPopulation{O <: AbstractOrganism} <: AbstractEntity end

# a mode of interaction between entities
abstract type AbstractDomain end

# a means of determining which entities interact 
abstract type AbstractRecipe end

# specifies (1) a set of entities or their keys, (2) the domain in which they interact
abstract type AbstractMix{T, D <: AbstractDomain} end

# an outcome of an interaction
abstract type AbstractOutcome end

# statistics associated with an entity
abstract type AbstractStatistics end

# driver
abstract type AbstractCoevolution <: AbstractEntity end

## Global Constants

In [3]:
Key = String
KEY_SPLIT_TOKEN = "-"

"-"

## Domain

In [4]:
abstract type NumbersGame <: AbstractDomain end
    
struct NGGradient <: NumbersGame end

function NGGradient(cfg::NamedTuple)
    NGGradient()
end

struct NGFocusing <: NumbersGame end

function NGFocusing(cfg::NamedTuple)
    NGFocusing()
end

struct NGRelativism <: NumbersGame end

function NGRelativism(cfg::NamedTuple)
    NGRelativism()
end

NGRelativism

## BasicGeno

In [5]:
struct BasicGeno{G, D <: AbstractDomain} <: AbstractGenotype
    key::Key
    genes::G
    stats::Set{AbstractStatistics}
    BasicGeno{G, D}(key, genes) where {G, D <: AbstractDomain} = new(key, genes, Set())
end

function BasicGeno{Vector{Bool}, D}(key::Key, val::Bool, width::Int) where {D <: AbstractDomain}
    bits = Bool[val for _ in 1:width]
    BasicGeno{Vector{Bool}, D}(key, bits)
end

function BasicGeno{Vector{Bool}, D}(key::Key, rng::AbstractRNG, width::Int) where {D <: AbstractDomain}
    bits = rand(rng, Bool, width)
    BasicGeno{Vector{Bool}, D}(key, bits)
end

function BasicGeno{Vector{Bool}, D}(key::Key, cfg::NamedTuple) where {D <: AbstractDomain}
    if haskey(cfg, :genome_default_val)
        BasicGeno{Vector{Bool}, D}(key, cfg.genome_default_val, cfg.genome_width)
    elseif haskey(cfg, :rng)
        BasicGeno{Vector{Bool}, D}(key, cfg.rng, cfg.genome_width)
    else
        error("Invalid config for genomes")
    end
end

In [6]:
T = Vector{Bool}
D = NumbersGame
G = BasicGeno{T, D}

# genome initialization with default value 0
cfg = (genome_default_val = false, genome_width = 10)
geno_zeros = G("AllZeros", cfg)
println(geno_zeros)

# genome initialization with default value 1
cfg = (genome_default_val = true, genome_width = 15)
geno_ones = G("AllOnes", cfg)
println(geno_ones)

# genome initialization with random values
cfg = (rng = StableRNG(123), genome_width = 20)
geno_rand = G("Random", cfg)
println(geno_rand)

BasicGeno{Vector{Bool}, NumbersGame}("AllZeros", Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Set{AbstractStatistics}())
BasicGeno{Vector{Bool}, NumbersGame}("AllOnes", Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Set{AbstractStatistics}())
BasicGeno{Vector{Bool}, NumbersGame}("Random", Bool[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1], Set{AbstractStatistics}())


## BasicPheno

In [7]:
struct BasicPheno{T, D <: AbstractDomain} <: AbstractPhenotype
    key::Key
    traits::T
    stats::Set{AbstractStatistics}
    BasicPheno{T, D}(key, traits) where {T, D} = new(key, traits, Set())
end

function BasicPheno{Int, NumbersGame}(key::Key, geno::BasicGeno{Vector{Bool}}, cfg::NamedTuple)
    BasicPheno{Int, NumbersGame}(key, sum(geno.genes))
end

function BasicPheno{Vector{Int}, NumbersGame}(key::Key, geno::BasicGeno{Vector{Bool}}, subvector_width::Int)
    if mod(length(geno.genes), subvector_width) != 0
        error("Invalid subvector width for given genome width")
    end
    traits = [sum(part) for part in Iterators.partition(geno.genes, subvector_width)]
    BasicPheno{Vector{Int}, NumbersGame}(key, traits)
end

function BasicPheno{Vector{Int}, NumbersGame}(key::Key, geno::BasicGeno{Vector{Bool}}, cfg::NamedTuple)
    BasicPheno{Vector{Int}, NumbersGame}(key, geno, cfg.subvector_width)
end

In [8]:
cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100)

T = Vector{Bool}
D = NumbersGame
G = BasicGeno{T, D}
geno_rand = G("RandGeno", cfg)

T = Int
D = NumbersGame
P = BasicPheno{T, D}
pheno_int = P("IntPheno", geno_rand, cfg)

T = Vector{Int}
D = NumbersGame
P = BasicPheno{T, D}
pheno_vec = P("VecPheno", geno_rand, cfg)

println(pheno_int)
println(pheno_vec)

BasicPheno{Int64, NumbersGame}("IntPheno", 47, Set{AbstractStatistics}())
BasicPheno{Vector{Int64}, NumbersGame}("VecPheno", [8, 5, 7, 5, 5, 2, 6, 1, 5, 3], Set{AbstractStatistics}())


## BasicOrg

In [9]:
struct BasicOrg{G, P} <: AbstractOrganism{G, P}
    key::Key
    geno::G
    pheno::P
    outcomes::Set{AbstractOutcome}
    stats::Set{AbstractStatistics}
    BasicOrg{G, P}(key, geno, pheno) where {G, P} = new(key, geno, pheno, Set(), Set())
end

function BasicOrg{G, P}(key::String, cfg::NamedTuple) where {G <: AbstractGenotype, P <: AbstractPhenotype}
    geno = G(key, cfg)
    pheno = P(key, geno, cfg)
    BasicOrg{G, P}(key, geno, pheno)
end

In [10]:
cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100)

T = Vector{Bool}
D = NumbersGame
G = BasicGeno{T, D}

T = Vector{Int}
D = NumbersGame
P = BasicPheno{T, D}

org = BasicOrg{G, P}("Organism", cfg)
println(org)

BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Vector{Int64}, NumbersGame}}("Organism", BasicGeno{Vector{Bool}, NumbersGame}("Organism", Bool[1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0], Set{AbstractStatistics}()), BasicPheno{Vector{Int64}, NumbersGame}("Organism", [8, 5, 7, 5, 5, 2, 6, 1, 5, 3], Set{AbstractStatistics}()), Set{AbstractOutcome}(), Set{AbstractStatistics}())


## BasicPop

In [11]:
struct BasicPop{O <: AbstractOrganism} <: AbstractPopulation{O}
    key::String
    orgs::Set{O}
    stats::Set{AbstractStatistics}
    BasicPop{O}(key, orgs) where {O <: AbstractOrganism} = new{O}(key, orgs, Set())
end

function BasicPop{O}(key::String, n_orgs::Int, cfg::NamedTuple) where {O <: AbstractOrganism}
    orgs = Set{O}()
    for i in 1:n_orgs
        orgkey = join([key, i], KEY_SPLIT_TOKEN)  
        org = O(orgkey, cfg)
        push!(orgs, org)
    end
    BasicPop{O}(key, orgs)
end

function BasicPop{O}(key::String, cfg::NamedTuple) where {O <: AbstractOrganism}
    BasicPop{O}(key, cfg.n_orgs, cfg)
end

function Dict{String, AbstractPhenotype}(pop::AbstractPopulation)
    Dict{String, AbstractPhenotype}([org.key => org.pheno for org in pop.orgs])
end

In [12]:
cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100, n_orgs = 10)
T = Vector{Bool}
D = NumbersGame
G = BasicGeno{T, D}

T = Vector{Int}
D = NumbersGame
P = BasicPheno{T, D}

O = BasicOrg{G, P}

pop = BasicPop{O}("test", cfg)
println(pop)

BasicPop{BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Vector{Int64}, NumbersGame}}}("test", Set(BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Vector{Int64}, NumbersGame}}[BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Vector{Int64}, NumbersGame}}("test-8", BasicGeno{Vector{Bool}, NumbersGame}("test-8", Bool[0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0], Set{AbstractStatistics}()), BasicPheno{Vector{Int64}, NumbersGame}("test-8", [5, 6, 5, 4, 6, 6, 7, 6, 9, 5], Set{AbstractStatistics}()), Set{AbstractOutcome}(), Set{AbstractStatistics}()), BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Vector{Int64}, NumbersGame}}("test-10", BasicGeno{Vector{Bool}, NumbersGame}("test-10", Bool[1, 0, 0, 0, 0,

In [13]:
@test length(pop.orgs) == 10
@test typeof(pop.orgs) == Set{BasicOrg{BasicGeno{Vector{Bool}, NumbersGame},
                                       BasicPheno{Vector{Int}, NumbersGame}}}

Test Passed

## PairMix

In [14]:
struct PairMix{T, D <: AbstractDomain} <: AbstractMix{T, D}
    subject::T
    test::T
end

function PairMix{String, D}(subject::AbstractOrganism, test::AbstractOrganism) where {D <: AbstractDomain}
    PairMix{String, D}(subject.key, test.key)
end

function PairMix{P, D}(subject::AbstractOrganism, test::AbstractOrganism) where {P <: AbstractPhenotype, D <: AbstractDomain}
    PairMix{P, D}(subject.pheno, test.pheno)
end

## Recipes

In [37]:
struct SampleRecipe <: AbstractRecipe
    rng::AbstractRNG
    n_samples::Int
    n_processes::Int
end

function SampleRecipe(cfg::NamedTuple)
    SampleRecipe(cfg.rng, cfg.n_samples, cfg.n_processes)
end

cfg = (rng = StableRNG(123), n_samples = 10, n_processes = 1)
@test SampleRecipe(cfg).n_samples == 10

Test Passed

In [16]:
d1 = Dict([1 => "a", 2 => "b"])
d2 = Dict([3 => "c", 4 => "d"])
reduce(merge, [d1, d2])

Dict{Int64, String} with 4 entries:
  4 => "d"
  2 => "b"
  3 => "c"
  1 => "a"

## Dictionaries of Descriptions
get all phenotypes/genotypes

In [17]:
function Dict{String, P}(pop::AbstractPopulation) where {P <: AbstractPhenotype}
    Dict([org.key => org.pheno for org in pop.orgs])
end

function Dict{String, G}(pop::AbstractPopulation) where {G <: AbstractGenotype}
    Dict([org.key => org.geno for org in pop.orgs])
end

function Dict{String, D}(coev::AbstractCoevolution) where {D <: AbstractDescription}
    reduce(merge, [Dict{String, D}(pop) for pop in coev.pops])
end



In [40]:
cfg = (rng = StableRNG(123), n_orgs = 10, genome_width = 100, n_samples = 10, n_processes = 1)
pop1 = BasicPop{O}("test1", cfg)
Dict{String, BasicGeno}(pop1)

Dict{String, BasicGeno{Vector{Bool}, NumbersGame}} with 10 entries:
  "test1-4"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-4", Bool[0, 0, 1, 0…
  "test1-8"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-8", Bool[0, 1, 1, 0…
  "test1-10" => BasicGeno{Vector{Bool}, NumbersGame}("test1-10", Bool[1, 0, 0, …
  "test1-9"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-9", Bool[1, 1, 0, 1…
  "test1-5"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-5", Bool[0, 0, 0, 0…
  "test1-7"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-7", Bool[1, 0, 0, 0…
  "test1-3"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-3", Bool[0, 0, 0, 0…
  "test1-2"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-2", Bool[1, 1, 1, 1…
  "test1-1"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-1", Bool[1, 0, 1, 1…
  "test1-6"  => BasicGeno{Vector{Bool}, NumbersGame}("test1-6", Bool[0, 0, 1, 1…

In [41]:
pop1 = BasicPop{O}("test1", cfg)
Dict{String, BasicPheno}(pop1)

Dict{String, BasicPheno{Int64, NumbersGame}} with 10 entries:
  "test1-4"  => BasicPheno{Int64, NumbersGame}("test1-4", 51, Set{AbstractStati…
  "test1-8"  => BasicPheno{Int64, NumbersGame}("test1-8", 50, Set{AbstractStati…
  "test1-9"  => BasicPheno{Int64, NumbersGame}("test1-9", 54, Set{AbstractStati…
  "test1-10" => BasicPheno{Int64, NumbersGame}("test1-10", 55, Set{AbstractStat…
  "test1-3"  => BasicPheno{Int64, NumbersGame}("test1-3", 50, Set{AbstractStati…
  "test1-7"  => BasicPheno{Int64, NumbersGame}("test1-7", 48, Set{AbstractStati…
  "test1-5"  => BasicPheno{Int64, NumbersGame}("test1-5", 51, Set{AbstractStati…
  "test1-2"  => BasicPheno{Int64, NumbersGame}("test1-2", 56, Set{AbstractStati…
  "test1-1"  => BasicPheno{Int64, NumbersGame}("test1-1", 49, Set{AbstractStati…
  "test1-6"  => BasicPheno{Int64, NumbersGame}("test1-6", 53, Set{AbstractStati…

In [20]:
function PairMix{String, D}(o1::AbstractOrganism, o2::AbstractOrganism) where {D <: AbstractDomain}
    PairMix{String, D}(o1.key, o2.key)
end

function PairMix(mix::PairMix{String, D}, phenos::Dict{String, P}) where {P <: AbstractPhenotype, D <: AbstractDomain}
    PairMix{P, D}(phenos[mix.solution], phenos[mix.test])
end


PairMix

In [21]:
cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100, n_orgs = 10)
T = Vector{Bool}
D = NumbersGame
G = BasicGeno{T, D}

T = Vector{Int}
D = NumbersGame
P = BasicPheno{T, D}

O = BasicOrg{G, P}

pop1 = BasicPop{O}("test1", cfg)
d1 = Dict{String, AbstractPhenotype}(pop1)
pop2 = BasicPop{O}("test2", cfg)
d2 = Dict{String, AbstractPhenotype}(pop2)

o1 = rand(cfg.rng, pop1.orgs)
o2 = rand(cfg.rng, pop2.orgs)

PairMix{String, NGGradient}(o1, o2)

PairMix{String, NGGradient}("test1-5", "test2-8")

In [22]:
function DummyPop(key::String="Test")
    cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100, n_orgs = 10)
    T = Vector{Bool}
    D = NumbersGame
    G = BasicGeno{T, D}

    T = Vector{Int}
    D = NumbersGame
    P = BasicPheno{T, D}

    O = BasicOrg{G, P}
    BasicPop{O}(key, cfg)
end

DummyPop (generic function with 2 methods)

In [23]:
function test(x, R::Type) 
    print(typeof(R))
    print(R <: Real)
    R(x) 
end 
test(5, Float64)

DataTypetrue

5.0

In [24]:
function Set{PairMix{String, D}}(subjects::AbstractPopulation, tests::AbstractPopulation, recipe::SampleRecipe) where {D <: AbstractDomain}
    mixes = Set{PairMix{String, D}}()
    for o1 in subjects.orgs
        for o2 in sample(recipe.rng, collect(tests.orgs), recipe.n_samples, replace=false)
            mix = PairMix{String, D}(o1, o2)
            push!(mixes, mix)
        end
    end
    mixes
end

function Set{PairMix{String, D}}(pops::Set{<:AbstractPopulation}, recipe::AbstractRecipe) where {D <: AbstractDomain, R <: AbstractRecipe}
    if length(pops) != 2
        error("Invalid number of populations for PairMix type.")
    end
    pop1, pop2 = pops
    mixes1 = Set{PairMix{String, D}}(pop1, pop2, recipe)
    mixes2 = Set{PairMix{String, D}}(pop2, pop1, recipe)
    union(mixes1, mixes2)
end

In [49]:
pops = Set([DummyPop("A"), DummyPop("B")])
cfg = (rng = StableRNG(123), n_samples = 5)
recipe = SampleRecipe(cfg)
mixes = Set{PairMix{String, NGGradient}}(pops, recipe)

LoadError: type NamedTuple has no field n_processes

## PairMix

In [26]:
function Set{String}(mix::M) where {M <: AbstractMix}
    Set([getfield(mix, fn) for fn in fieldnames(M)])
end

function Set{String}(mixes::Set{M}) where {M <: AbstractMix}
    union([Set{String}(mix) for mix in mixes]...)
end

In [27]:
function Set{Set{M}}(mixes::Set{<:AbstractMix}, n::Int) where {M <: AbstractMix}
    n_mix = div(length(mixes), n)
    alljobmixes = Set{M}[]
    m = collect(Iterators.partition(collect(mixes), n_mix))
    if length(m) > n
        excess = pop!(m)
        for i in eachindex(excess)
            push!(m[i], excess[i])
        end
    end
    Set([Set{M}(x) for x in m])
end
    

In [43]:
pops = Set([DummyPop("A"), DummyPop("B")])
cfg = (popkeys = Set(["A", "B"]), genome_width = 10, n_samples = 5, n_orgs = 10, rng = StableRNG(123), n_processes = 1)
recipe = SampleRecipe(cfg)
mixes = Set{PairMix{String, NGGradient}}(pops, recipe)
Set{Set{PairMix}}(mixes, 10)


Set{Set{PairMix}} with 10 elements:
  Set(PairMix[PairMix{String, NGGradient}("B-9", "A-9"), PairMix{String, NGGrad…
  Set(PairMix[PairMix{String, NGGradient}("B-5", "A-10"), PairMix{String, NGGra…
  Set(PairMix[PairMix{String, NGGradient}("A-3", "B-2"), PairMix{String, NGGrad…
  Set(PairMix[PairMix{String, NGGradient}("A-2", "B-10"), PairMix{String, NGGra…
  Set(PairMix[PairMix{String, NGGradient}("B-2", "A-8"), PairMix{String, NGGrad…
  Set(PairMix[PairMix{String, NGGradient}("A-9", "B-4"), PairMix{String, NGGrad…
  Set(PairMix[PairMix{String, NGGradient}("A-9", "B-10"), PairMix{String, NGGra…
  Set(PairMix[PairMix{String, NGGradient}("A-10", "B-1"), PairMix{String, NGGra…
  Set(PairMix[PairMix{String, NGGradient}("A-1", "B-10"), PairMix{String, NGGra…
  Set(PairMix[PairMix{String, NGGradient}("B-7", "A-9"), PairMix{String, NGGrad…

In [44]:
struct BasicCoev{P <: AbstractPopulation, R <: AbstractRecipe, M <: AbstractMix} <: AbstractCoevolution
    gen::Int
    pops::Set{P}
    mixes::Set{M}
    stats::Set{AbstractStatistics}
    BasicCoev{P, R, M}(gen, pops, mixes) where {P <: AbstractPopulation, R <: AbstractRecipe, M <: AbstractMix} = new{P, R, M}(gen, pops, mixes, Set())
end

function BasicCoev{P, R, M}(popkeys::Set{String}, cfg::NamedTuple) where {P <: AbstractPopulation, M <: AbstractMix, R <: AbstractRecipe}
    pops = Set([P(popkey, cfg) for popkey in popkeys])
    mixes = Set{M}(pops, R(cfg))
    BasicCoev{P, R, M}(1, pops, mixes)
end
function BasicCoev{P, R, M}(cfg::NamedTuple) where {P <: AbstractPopulation, M <: AbstractMix, R <: AbstractRecipe}
    BasicCoev{P, R, M}(cfg.popkeys, cfg)
end



In [45]:
cfg = (popkeys = Set(["A", "B"]), genome_width = 10, n_samples = 5, n_orgs = 10, rng = StableRNG(123), n_processes = 1)

T = Vector{Bool}
D = NumbersGame
# the genotype is a bitstring
G = BasicGeno{T, D}
# the phenotype is the sum of the 1s in the genotype
PH = BasicPheno{Int, D}
# organism is built from these two types
O = BasicOrg{G, PH}
# population is built from these orabisms
P = BasicPop{O}
# the game/domain is the simple "Greater Than" game
# the mix comprises pairs of organisms fated to play the numbers game
D = NGGradient
M = PairMix{String, D}
R = SampleRecipe
# 

coev = BasicCoev{P, R, M}(cfg)


BasicCoev{BasicPop{BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Int64, NumbersGame}}}, SampleRecipe, PairMix{String, NGGradient}}(1, Set(BasicPop{BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Int64, NumbersGame}}}[BasicPop{BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Int64, NumbersGame}}}("B", Set(BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Int64, NumbersGame}}[BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Int64, NumbersGame}}("B-8", BasicGeno{Vector{Bool}, NumbersGame}("B-8", Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 1], Set{AbstractStatistics}()), BasicPheno{Int64, NumbersGame}("B-8", 1, Set{AbstractStatistics}()), Set{AbstractOutcome}(), Set{AbstractStatistics}()), BasicOrg{BasicGeno{Vector{Bool}, NumbersGame}, BasicPheno{Int64, NumbersGame}}("B-2", BasicGeno{Vector{Bool}, NumbersGame}("B-2", Bool[1, 1, 1, 1, 0, 0, 0, 0, 0, 1], Set{AbstractStatistics}()), BasicPheno{Int64, NumbersGame}("B-2", 5, Set{AbstractStatistics}()), Set{Ab

In [46]:
Dict{String, AbstractPhenotype}(rand(coev.pops))
Dict{String, AbstractPhenotype}(coev)

Dict{String, AbstractPhenotype} with 20 entries:
  "B-9"  => BasicPheno{Int64, NumbersGame}("B-9", 5, Set{AbstractStatistics}())
  "A-4"  => BasicPheno{Int64, NumbersGame}("A-4", 5, Set{AbstractStatistics}())
  "A-6"  => BasicPheno{Int64, NumbersGame}("A-6", 7, Set{AbstractStatistics}())
  "B-8"  => BasicPheno{Int64, NumbersGame}("B-8", 1, Set{AbstractStatistics}())
  "B-5"  => BasicPheno{Int64, NumbersGame}("B-5", 5, Set{AbstractStatistics}())
  "A-7"  => BasicPheno{Int64, NumbersGame}("A-7", 3, Set{AbstractStatistics}())
  "B-2"  => BasicPheno{Int64, NumbersGame}("B-2", 5, Set{AbstractStatistics}())
  "B-6"  => BasicPheno{Int64, NumbersGame}("B-6", 2, Set{AbstractStatistics}())
  "A-9"  => BasicPheno{Int64, NumbersGame}("A-9", 5, Set{AbstractStatistics}())
  "B-4"  => BasicPheno{Int64, NumbersGame}("B-4", 5, Set{AbstractStatistics}())
  "A-1"  => BasicPheno{Int64, NumbersGame}("A-1", 7, Set{AbstractStatistics}())
  "A-8"  => BasicPheno{Int64, NumbersGame}("A-8", 7, Set{AbstractStatis

## PairOutcome

In [32]:
struct PairOutcome{T <: Real} <: AbstractOutcome
    solution::Key
    test::Key
    score::T
end

function Set{PairOutcome}(mixset::Set{PairMix{T, D}}, cfg::NamedTuple) where {T <: AbstractPhenotype, D <: AbstractDomain}
    reduce(union, [PairOutcome(mix.subject, mix.test, D(cfg)) for mix in mixset])
end

function PairOutcome(mix::PairMix{T, D}, cfg::NamedTuple) where {T <: AbstractPhenotype, D <: AbstractDomain}
    PairOutcome(mix.subject, mix.test, D(cfg))
end

function PairOutcome(subject::BasicPheno{Int}, test::BasicPheno{Int}, domain::NGGradient)
    PairOutcome(subject.key, test.key, subject.traits > test.traits)
end

function PairOutcome(subject::BasicPheno{Vector{Int}}, test::BasicPheno{Vector{Int}}, domain::NGGradient)
    PairOutcome(subject.key, test.key, sum(subject.traits) > sum(test.traits))
end

function PairOutcome(subject::BasicPheno{Vector{Int}}, test::BasicPheno{Vector{Int}}, domain::NGFocusing)
    v1, v2 = subject.traits, test.traits
    maxdiff, idx = findmax([abs(x1 - x2) for (x1, x2) in zip(v1, v2)])
    PairOutcome(subject.key, test.key, v1[idx] > v2[idx])
end

function PairOutcome(subject::BasicPheno{Vector{Int}}, test::BasicPheno{Vector{Int}}, domain::NGRelativism)
    v1, v2 = subject.traits, test.traits
    mindiff, idx = findmin([abs(x1 - x2) for (x1, x2) in zip(v1, v2)])
    PairOutcome(subject.key, test.key, v1[idx] > v2[idx])
end


PairOutcome

In [50]:
function Set{O}(coev::BasicCoev{P, R, M}, cfg::NamedTuple) where {O <: AbstractOutcome} 
    phenos = Dict{String, AbstractPhenotype}(coev)
    mixes = Set{M}(coev.pops, R(cfg))
    mixsets = Set{Set{M}}(mixes, cfg.n_processes)
    for mixset in mixsets
        
        
        O(mixset, phenos, D())
    end
    outcomes
    
    
end

In [51]:
cfg = (popkeys = Set(["A", "B"]), genome_width = 10, n_samples = 5, n_orgs = 10, rng = StableRNG(123), n_processes=10)
#Dict{String, AbstractPhenotype}(coev)
Set{PairOutcome}(coev, cfg)

LoadError: MethodError: no method matching PairOutcome(::Set{PairMix{String, NGGradient}}, ::Dict{String, AbstractPhenotype}, ::NGGradient)
[0mClosest candidates are:
[0m  PairOutcome([91m::BasicPheno{Int64, D} where D<:AbstractDomain[39m, [91m::BasicPheno{Int64, D} where D<:AbstractDomain[39m, ::NGGradient) at In[32]:15
[0m  PairOutcome([91m::BasicPheno{Vector{Int64}, D} where D<:AbstractDomain[39m, [91m::BasicPheno{Vector{Int64}, D} where D<:AbstractDomain[39m, ::NGGradient) at In[32]:19

In [35]:
cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100, n_orgs = 10)
T = Vector{Bool}
D = NumbersGame
G = BasicGeno{T, D}

T = Int
D = NumbersGame
P = BasicPheno{T, D}

O = BasicOrg{G, P}
org1 = O("org1", cfg)
org2 = O("org2", cfg)

D = NGGradient
phenomix = PairMix{BasicPheno, NGGradient}(org1, org2)
PairOutcome(phenomix, cfg)

PairOutcome{Bool}("org1", "org2", false)

In [36]:
function run_mixdemo(cfg::NamedTuple)
    T = Vector{Bool}
    D = NumbersGame
    G = BasicGeno{T, D}

    T = Vector{Int}
    D = NumbersGame
    P = BasicPheno{T, D}

    O = BasicOrg{G, P}

    org1, org2 = O("org1", cfg), O("org2", cfg)
    t1, t2 = org1.pheno.traits, org2.pheno.traits
    println("$(t1), sum = $(sum(t1))")
    println("$(t2), sum = $(sum(t2))")
    cfg = (heh = "hehe",)

    for D in subtypes(NumbersGame)
        phenomix = PairMix{P, D}(org1, org)
        o = PairOutcome(phenomix, cfg)
        println(D, " - ", o.score)
    end
end

run_mixdemo (generic function with 1 method)

In [54]:
cfg = (rng = StableRNG(123), subvector_width = 10, genome_width = 100, n_orgs = 10)
run_mixdemo(cfg)
cfg = (rng = StableRNG(42), subvector_width = 10, genome_width = 100, n_orgs = 10)
run_mixdemo(cfg)
cfg = (rng = StableRNG(67), subvector_width = 10, genome_width = 100, n_orgs = 10)
run_mixdemo(cfg)
cfg = (rng = StableRNG(3325), subvector_width = 10, genome_width = 100, n_orgs = 10)
run_mixdemo(cfg)

LoadError: UndefVarError: run_mixdemo not defined

In [22]:
v1 = [4, 4, 8, 6, 4, 2, 6, 5, 7, 3]
v2 = [9, 3, 1, 7, 5, 3, 5, 3, 8, 7]


d = Dict{Int, Vector{Bool}}()


for (diff, gt) in [(abs(x1 - x2), x1 > x2) for (x1, x2) in zip(v1, v2)]
    if haskey(d, diff)
        push!(d[diff], gt)
    else
        d[diff] = Bool[gt]
    end
end
d


Dict{Int64, Vector{Bool}} with 5 entries:
  5 => [0]
  4 => [0]
  7 => [1]
  2 => [1]
  1 => [1, 0, 0, 0, 1, 0]

In [ ]:

struct StatFeatures
    mean::Float64
    quartiles::Vector{Float64}
    var::Float64
    std::Float64
end

struct NGOrgStats <: AbstractStatistics
    subjective_fitness::Int
    objective_fitness::Int
end
struct NGPopStats <: AbstractStatistics
    obj_fitness::StatFeatures
    subj_fitness::StatFeatures
end
